This notebooks shows some details within the RefDNN model. CCLE dataset is used as an example here.

In [1]:
# import libraries
import os
import math
import numpy as np
import pandas as pd
import skopt
from datetime import datetime
import time
from sklearn.metrics import accuracy_score, average_precision_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.model_selection import StratifiedKFold, train_test_split
from refdnn.model import REFDNN
from refdnn.dataset import DATASET
from scipy.spatial.distance import pdist, squareform
import tensorflow as tf

In [2]:
os.environ['TF_CPP_MIN_VLOG_LEVEL'] = '1'
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'

In [3]:
# key arguments

global outputdir
global checkpointdir
global verbose

# A filepath of drug response data for TRAINING
responseFile = 'data/response_CCLE.csv'          
# A filepath of gene expression data for TRAINING
expressionFile = 'data/expression_CCLE.csv'       
# A filepath of fingerprint data for TRAINING
fingerprintFile = 'data/fingerprint_CCLE.csv'      
# A directory path for saving outputs (default:'output_1')
outputdir = 'output_2_CCLE'         
# A size of batch on training process. The small size is recommended if an available size of RAM is small (default: 64)
batchsize = 32 #b        
# Number of training steps on training process. It is recommended that the steps is larger than (numpairs /batchsize) (default: 5000)
numtrainingsteps  = 1000 #t
# Number of bayesian search for hyperparameter tuning (default: 20)
numbayesiansearch = 10 #s
# K for outer k-fold cross validation (default: 5)
outerkfold = 5 # k        
# L for inner l-fold cross validation (default: 3)
innerkfold = 3 # l       
# 0:No logging, 1:Basic logging to check process, 2:Full logging for debugging (default:1)
verbose = 2 # v         

In [4]:
# input data files:

# A filepath of drug response data for TRAINING
response = pd.read_csv(responseFile)        
# A filepath of gene expression data for TRAINING
expression = pd.read_csv(expressionFile) 
# A filepath of fingerprint data for TRAINING
fingerprint = pd.read_csv(fingerprintFile) 

In [14]:
# the DATASET class

class DATASET:
    def __init__(self, drfile, gefile, fpfile):
        ## 1. Read data
        ## 1-1) Gene expression
        self.ge = pd.read_csv(gefile, sep=',', index_col=0)
        ## 1-2) Drug response
        self.dr = pd.read_csv(drfile, sep=',', dtype='str')
        self.DRUGKEY = self.dr.columns[0]
        self.CELLKEY = self.dr.columns[1]
        self.LABELKEY = self.dr.columns[2]
        ## 1-3) Fingerprint
        self.fp = pd.read_csv(fpfile, sep=',', index_col=0).transpose()
        
        ## 2. Preprocessing
        ## 2-1) Find targets
        target_drugs = self._find_target_drugs()
        target_cells = self._find_target_cells()
        ## 2-2) Filter data
        self.ge = self.ge.filter(target_cells)
        self.dr = self.dr[self._get_target_idx(target_drugs, target_cells)]
        ## 2-3) Label string to integer
        idx = self.dr[self.LABELKEY] == 'Resistance'
        self.dr[self.LABELKEY][idx] = 1
        self.dr[self.LABELKEY][~idx] = 0
        self.dr[self.LABELKEY] = self.dr[self.LABELKEY].astype(np.uint8)
        
        ## 3. Structural Similarity Profile
        self.SSP = self._make_SSP()

    def __len__(self):
        return len(self.dr)
        
    def get_drugs(self, unique=False):
        return self._get_series(self.DRUGKEY, unique)
        
    def get_cells(self, unique=False):
        return self._get_series(self.CELLKEY, unique)
        
    def get_labels(self, unique=False):
        return self._get_series(self.LABELKEY, unique)
        
    def get_genes(self):
        return self.ge.index.values
        
    def get_exprs(self):
        return self.ge.values.T # per cell
    
    def make_xdata(self, idx=None):
        if idx is None:
            cells = self.get_cells()
        else:
            cells = self.get_cells()[idx]
        return np.array([self.ge[cell].values for cell in cells], dtype=np.float32)
    
    def make_ydata(self, idx=None):
        if idx is None:
            labels = self.get_labels()
        else:
            labels = self.get_labels()[idx]
        return np.expand_dims(np.array(labels, dtype=np.uint8), axis=-1)
    
    def make_sdata(self, base_drugs, idx=None):
        SSP = self.SSP.filter(base_drugs, axis='index')
        if idx is None:
            drugs = self.get_drugs()
        else:
            drugs = self.get_drugs()[idx]
        return np.array([SSP[drug].values for drug in drugs], dtype=np.float32)
        
    def make_idata(self, base_drugs, idx=None):
        drug2idx = {drug:i for i, drug in enumerate(base_drugs)}
        if idx is None:
            drugs = self.get_drugs()
        else:
            drugs = self.get_drugs()[idx]
        return np.array([drug2idx[drug] for drug in drugs], dtype=np.int64)
    
    def _make_SSP(self):
        ssp_mat = 1. - squareform(pdist(self.fp.values.T, 'jaccard'))
        return pd.DataFrame(ssp_mat, index=self.fp.columns, columns=self.fp.columns)
        
    def _get_series(self, KEY, unique):
        if unique:
            return np.sort(self.dr[KEY].unique(), kind='mergesort')
        else:
            return self.dr[KEY].values
    
    def _find_target_drugs(self):
        target_drugs = []
        self.drug2fp = {}
        for drugname in self.dr[self.DRUGKEY].unique():
            if drugname in self.fp:
                target_drugs.append(drugname)
                self.drug2fp[drugname] = self.fp[drugname].astype(np.uint8).astype(bool)
        return target_drugs
        
    def _find_target_cells(self):
        return self.ge.columns.intersection(self.dr[self.CELLKEY].unique())

    def _get_target_idx(self, target_drugs, target_cells):
        idx_drugs = self.dr[self.DRUGKEY].isin(target_drugs)
        idx_cells = self.dr[self.CELLKEY].isin(target_cells)
        return idx_drugs & idx_cells    

We first read data into a DATASET instance:

In [15]:
########################################################
## 1. Read data
########################################################    

global dataset
dataset = DATASET(responseFile, expressionFile, fingerprintFile)
if verbose > 0:
    print('[DATA] NUM_PAIRS: {}'.format(len(dataset)))
    print('[DATA] NUM_DRUGS: {}'.format(len(dataset.get_drugs(unique=True))))
    print('[DATA] NUM_CELLS: {}'.format(len(dataset.get_cells(unique=True))))
    print('[DATA] NUM_GENES: {}'.format(len(dataset.get_genes())))
    print('[DATA] NUM_SENSITIVITY: {}'.format(np.count_nonzero(dataset.get_labels()==0)))
    print('[DATA] NUM_RESISTANCE: {}'.format(np.count_nonzero(dataset.get_labels()==1)))

[DATA] NUM_PAIRS: 5724
[DATA] NUM_DRUGS: 12
[DATA] NUM_CELLS: 491
[DATA] NUM_GENES: 18926
[DATA] NUM_SENSITIVITY: 2322
[DATA] NUM_RESISTANCE: 3402


### Play with some methods of the DATASET class:

In [16]:
dataset.get_drugs(unique=True)

array(['Erlotinib', 'Lapatinib', 'Nilotinib', 'Nutlin-3a (-)',
       'PD0325901', 'PHA-665752', 'PLX-4720', 'Paclitaxel', 'Palbociclib',
       'Selumetinib', 'Sorafenib', 'Tanespimycin'], dtype=object)

In [17]:
dataset.get_drugs()

array(['Nilotinib', 'Nilotinib', 'Nilotinib', ..., 'Erlotinib',
       'Erlotinib', 'Erlotinib'], dtype=object)

In [18]:
dataset.get_cells(unique=True)

array(['1321N1_CENTRAL_NERVOUS_SYSTEM', '22RV1_PROSTATE',
       '42MGBA_CENTRAL_NERVOUS_SYSTEM', '5637_URINARY_TRACT',
       '639V_URINARY_TRACT', '697_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
       '769P_KIDNEY', '786O_KIDNEY', '8305C_THYROID', '8505C_THYROID',
       '8MGBA_CENTRAL_NERVOUS_SYSTEM', 'A172_CENTRAL_NERVOUS_SYSTEM',
       'A204_SOFT_TISSUE', 'A2058_SKIN', 'A253_SALIVARY_GLAND',
       'A2780_OVARY', 'A375_SKIN', 'A549_LUNG', 'A673_BONE',
       'ACHN_KIDNEY', 'ALLSIL_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
       'AMO1_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'AN3CA_ENDOMETRIUM',
       'ASPC1_PANCREAS', 'AU565_BREAST', 'AZ521_STOMACH', 'BCPAP_THYROID',
       'BDCM_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'BFTC909_KIDNEY',
       'BL41_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE',
       'BL70_HAEMATOPOIETIC_AND_LYMPHOID_TISSUE', 'BT20_BREAST',
       'BT474_BREAST', 'BT549_BREAST', 'BXPC3_PANCREAS',
       'C2BBE1_LARGE_INTESTINE', 'C32_SKIN', 'C3A_LIVER', 'CAKI2_KIDNEY',
       'CAL12T_L

In [19]:
dataset.get_genes()

array([    0,     1,     2, ..., 18923, 18924, 18925])

In [20]:
dataset.get_labels()

array([1, 1, 1, ..., 1, 1, 1], dtype=uint8)

In [21]:
dataset.get_exprs()

array([[4.33, 4.35, 5.29, ..., 8.23, 4.88, 3.66],
       [3.82, 4.08, 3.71, ..., 8.04, 4.95, 3.6 ],
       [3.91, 4.49, 3.75, ..., 8.8 , 4.89, 3.96],
       ...,
       [5.42, 8.27, 7.53, ..., 9.6 , 4.87, 3.71],
       [4.1 , 7.8 , 3.8 , ..., 8.32, 4.35, 3.89],
       [4.87, 7.43, 3.87, ..., 5.35, 5.  , 3.82]])

**Note:** For the Bayesian Optimization process, we need to define the hyperparameter space first and perform the Bayesian search. Here our purpose is to show how the model is trained, so let's ignore the hyperparameter part, and directly go with a given set of hyperparamters.

In [34]:
# this will be an example set of parameters we decide to use after the Bayesian optimization
BEST_HIDDEN_UNITS = 128
BEST_LEARNING_RATE_FTRL = 0.1
BEST_LEARNING_RATE_ADAM = 0.014530495141666015
BEST_L1_REGULARIZATION_STRENGTH = 100.0
BEST_L2_REGULARIZATION_STRENGTH = 0.001

Split the data for outer loop of cross validation:

In [23]:
global fitness_step
global fitness_idx_train
global fitness_idx_test
global innerkfold
global batchsize
global numtrainingsteps
    
## 3-1) init lists for metrics
ACCURACY_outer = []
AUCROC_outer = []
AUCPR_outer = []

kf = StratifiedKFold(n_splits=outerkfold, shuffle=True)

We can take a look at the splits:

In [24]:
splits = []
for k, (idx_train, idx_test) in enumerate(kf.split(X=np.zeros(len(dataset)), y=dataset.get_drugs())): 
    print(f'split {k}: train {len(idx_train)}, test {len(idx_test)}')
    splits.append((idx_train, idx_test))

split 0: train 4579, test 1145
split 1: train 4579, test 1145
split 2: train 4579, test 1145
split 3: train 4579, test 1145
split 4: train 4580, test 1144


In [25]:
# the indexes of each split
splits[0]

(array([   0,    1,    3, ..., 5721, 5722, 5723]),
 array([   2,    9,   12, ..., 5710, 5713, 5716]))

In [26]:
splits[1]

(array([   0,    1,    2, ..., 5720, 5721, 5723]),
 array([   3,    6,   14, ..., 5715, 5717, 5722]))

In [27]:
splits[2]

(array([   0,    2,    3, ..., 5718, 5721, 5722]),
 array([   1,    4,    8, ..., 5719, 5720, 5723]))

In [28]:
splits[3]

(array([   1,    2,    3, ..., 5720, 5722, 5723]),
 array([   0,   10,   16, ..., 5708, 5718, 5721]))

In [29]:
splits[4]

(array([   0,    1,    2, ..., 5721, 5722, 5723]),
 array([   5,    7,   19, ..., 5703, 5707, 5714]))

Let's just use the first split as an example:

In [30]:
#for k, (idx_train, idx_test) in enumerate(kf.split(X=np.zeros(len(dataset)), y=dataset.get_drugs())):    

# for a certain split:
idx_train, idx_test =  splits[0]
fitness_step = 1
fitness_idx_train, fitness_idx_test = splits[0]

In [31]:
len(idx_train)

4579

In [32]:
len(idx_test)

1145

Further split the train data into train and validation, for model training:

In [35]:
idx_train_train, idx_train_valid = train_test_split(idx_train, test_size=0.2, stratify=dataset.get_drugs()[idx_train])

In [36]:
len(idx_train_train)

3663

In [37]:
idx_train_train

array([ 884, 2155, 3273, ...,  148, 3977, 3133])

In [38]:
len(idx_train_valid)

916

In [39]:
idx_train_valid

array([5494, 3212, 2912, 1252, 5377,  975, 2075,  922, 4371, 2728, 3529,
       3667, 1480, 5379, 1677, 5086, 1476,  886,  460, 4574, 2174, 3903,
        561,  934, 4978, 1734, 1013, 3539,  980, 4111, 3846, 4498,  601,
        463, 5539, 4255, 3291, 1981, 1576, 4310, 3860, 1020, 1864, 3208,
       3932, 1906,  476,  579,  565, 2226, 1484, 4925, 3459,  263, 3131,
        644, 4641, 4214, 2514, 4916, 5609, 4250, 4170,  353, 4654, 3232,
        547, 3491, 4962, 3145, 1525, 2699, 2824, 4474, 4307, 1533, 5472,
        271,  633,  484, 1666,   76, 3384,  261, 3155, 3798, 5222, 3015,
       5345, 1179, 1336, 4194, 4295,  755, 2906, 1337, 3560, 1853, 4960,
       2279, 1288, 3898, 4027, 5143, 2897,  879,  341, 2270, 2100, 4131,
        893, 1231, 1630, 1396, 2743, 3631, 1625, 1398,  860, 2649, 4769,
       2408, 4832, 2453, 3863, 1301, 1259, 3355, 1436, 2627, 2737,  844,
       2386, 3281, 4456, 1172, 4836, 3140, 1350, 4342, 2606,   38, 5291,
       4325, 1307, 4368, 4683, 3879, 2859, 4219, 18

This is the "reference drugs". In the default, it's just all of the unique drugs in the dataset:

In [40]:
base_drugs = np.unique(dataset.get_drugs()[idx_train_train])
base_drugs

array(['Erlotinib', 'Lapatinib', 'Nilotinib', 'Nutlin-3a (-)',
       'PD0325901', 'PHA-665752', 'PLX-4720', 'Paclitaxel', 'Palbociclib',
       'Selumetinib', 'Sorafenib', 'Tanespimycin'], dtype=object)

### Prepare the data for the model:

In [41]:
X_train = dataset.make_xdata(idx_train_train)
S_train = dataset.make_sdata(base_drugs, idx_train_train)
I_train = dataset.make_idata(base_drugs, idx_train_train)
Y_train = dataset.make_ydata(idx_train_train)

In [42]:
# each row is a cell line (of the cellline-drug pair)
# 3663 pairs, 18926 genes
print(X_train.shape)
X_train

(3663, 18926)


array([[3.9 , 4.28, 3.92, ..., 8.97, 5.19, 7.35],
       [4.26, 4.43, 3.76, ..., 9.1 , 5.11, 4.12],
       [4.17, 4.71, 7.67, ..., 8.49, 5.18, 4.43],
       ...,
       [4.15, 6.96, 8.83, ..., 6.25, 5.2 , 5.21],
       [3.91, 4.57, 3.88, ..., 8.12, 5.25, 3.98],
       [3.8 , 7.91, 3.93, ..., 7.84, 5.11, 4.3 ]], dtype=float32)

In [43]:
# each row is a drug (of the cellline-drug pair)
# 3663 pairs, similarity to the 12 drugs
print(S_train.shape)
S_train

(3663, 12)


array([[0.30208334, 0.3431007 , 0.3363585 , ..., 0.3378076 , 0.27724552,
        1.        ],
       [0.369994  , 0.43024555, 0.36560366, ..., 0.35072142, 0.3653484 ,
        0.36780384],
       [0.3039711 , 0.3170577 , 0.33147633, ..., 0.521875  , 0.29923075,
        0.29837704],
       ...,
       [0.44382802, 0.4426732 , 1.        , ..., 0.44885993, 0.3896747 ,
        0.3363585 ],
       [0.3310185 , 0.37929183, 0.35778782, ..., 0.34175825, 0.33844268,
        0.4818653 ],
       [0.3039711 , 0.3170577 , 0.33147633, ..., 0.521875  , 0.29923075,
        0.29837704]], dtype=float32)

In [44]:
# a vector of the indexes of the drugs
print(I_train.shape)
I_train

(3663,)


array([11,  3,  4, ...,  2,  7,  4])

In [45]:
# the labels (resistant or sensitive)
print(Y_train.shape)
Y_train

(3663, 1)


array([[0],
       [0],
       [0],
       ...,
       [1],
       [0],
       [1]], dtype=uint8)

In [46]:
X_valid = dataset.make_xdata(idx_train_valid)
S_valid = dataset.make_sdata(base_drugs, idx_train_valid)
I_valid = dataset.make_idata(base_drugs, idx_train_valid)
Y_valid = dataset.make_ydata(idx_train_valid)

In [47]:
X_test = dataset.make_xdata(idx_test)
S_test = dataset.make_sdata(base_drugs, idx_test)
Y_test = dataset.make_ydata(idx_test)

In [48]:
checkpoint_dir = './temp'
if not os.path.exists(checkpoint_dir):
    os.mkdir(checkpoint_dir)
checkpoint_path = "RefDNN_cv_outer.ckpt"
checkpoint_path = os.path.join('./temp', checkpoint_path)

### Create a model:

In [49]:
## 3-5) Create a model using the best parameters
clf = REFDNN(hidden_units=BEST_HIDDEN_UNITS,
             learning_rate_ftrl=BEST_LEARNING_RATE_FTRL,
             learning_rate_adam=BEST_LEARNING_RATE_ADAM,
             l1_regularization_strength=BEST_L1_REGULARIZATION_STRENGTH,
             l2_regularization_strength=BEST_L2_REGULARIZATION_STRENGTH,
             batch_size=batchsize,
             training_steps=numtrainingsteps,
             checkpoint_path=checkpoint_path)

In [50]:
print(clf)

### Fit a model:

In [51]:
## 3-6) Fit a model
history = clf.fit(X_train, S_train, I_train, Y_train,
                  X_valid, S_valid, I_valid, Y_valid,
                  verbose=verbose)

[RefDNN] [00100] LOSS_train=0.41091 | LOSS_valid=0.60444 | ACC_valid=0.872 | PRECISION_valid=0.862 | RECALL_valid=0.921 | (1.972 sec)
[RefDNN] [CHECKPOINT] Model is saved in: ./temp/RefDNN_cv_outer.ckpt
[RefDNN] [00200] LOSS_train=0.48195 | LOSS_valid=0.56305 | ACC_valid=0.896 | PRECISION_valid=0.870 | RECALL_valid=0.968 | (0.646 sec)
[RefDNN] [CHECKPOINT] Model is saved in: ./temp/RefDNN_cv_outer.ckpt
[RefDNN] [00300] LOSS_train=0.46646 | LOSS_valid=0.52509 | ACC_valid=0.895 | PRECISION_valid=0.867 | RECALL_valid=0.969 | (0.617 sec)
[RefDNN] [CHECKPOINT] Model is saved in: ./temp/RefDNN_cv_outer.ckpt
[RefDNN] [00400] LOSS_train=0.40961 | LOSS_valid=1.12804 | ACC_valid=0.854 | PRECISION_valid=0.814 | RECALL_valid=0.975 | (0.613 sec)
[RefDNN] [00500] LOSS_train=0.20285 | LOSS_valid=1.13649 | ACC_valid=0.794 | PRECISION_valid=0.747 | RECALL_valid=0.981 | (0.551 sec)
[RefDNN] [00600] LOSS_train=0.28100 | LOSS_valid=0.55395 | ACC_valid=0.756 | PRECISION_valid=0.711 | RECALL_valid=0.984 | (

# How exactly was the model trained?

Let's look into some more details of the model here. Here I copy some code out from the RefDNN class.

In [78]:
# parameters
hidden_units=BEST_HIDDEN_UNITS
learning_rate_ftrl=BEST_LEARNING_RATE_FTRL
learning_rate_adam=BEST_LEARNING_RATE_ADAM
l1_regularization_strength=BEST_L1_REGULARIZATION_STRENGTH
l2_regularization_strength=BEST_L2_REGULARIZATION_STRENGTH
batch_size=batchsize
training_steps=numtrainingsteps
verbose=verbose

## constant
epsilon = 1e-5
threshold = 0.5

In [79]:
## data information
_X_shape = (None, X_train.shape[1]) # number of genes
_S_shape = (None, S_train.shape[1]) # number of drugs

print(_X_shape)
print(_S_shape)

(None, 18926)
(None, 12)


### Start a tensorflow session here:

In [80]:
## tf.Session
#self._open_session()
tf.reset_default_graph()
sess = tf.Session()

### prepare the placeholder and dataset

In [81]:
## tf.placeholder
X_train_PH = tf.placeholder(shape=_X_shape, dtype=tf.float32, name='X_train_PH')
S_train_PH = tf.placeholder(shape=_S_shape, dtype=tf.float32, name='S_train_PH')
I_train_PH = tf.placeholder(shape=(None,), dtype=tf.int32, name='I_train_PH')
Y_train_PH = tf.placeholder(shape=(None,1), dtype=tf.uint8, name='Y_train_PH')

X_valid_PH = tf.placeholder(shape=_X_shape, dtype=tf.float32, name='X_valid_PH')
S_valid_PH = tf.placeholder(shape=_S_shape, dtype=tf.float32, name='S_valid_PH')
I_valid_PH = tf.placeholder(shape=(None,), dtype=tf.int32, name='I_valid_PH')
Y_valid_PH = tf.placeholder(shape=(None,1), dtype=tf.uint8, name='Y_valid_PH')

In [82]:
X_train_PH

<tf.Tensor 'X_train_PH:0' shape=(?, 18926) dtype=float32>

In [83]:
## tf.dataset
dataset_train = tf.data.Dataset.from_tensor_slices((X_train_PH, S_train_PH, I_train_PH, Y_train_PH))
dataset_train = dataset_train.repeat()
dataset_train = dataset_train.shuffle(buffer_size=10000, seed=2019)
dataset_train = dataset_train.batch(batch_size, drop_remainder=False)
dataset_train.prefetch(2 * batch_size)

dataset_valid = tf.data.Dataset.from_tensor_slices((X_valid_PH, S_valid_PH, I_valid_PH, Y_valid_PH))
dataset_valid = dataset_valid.repeat()
dataset_valid = dataset_valid.batch(batch_size, drop_remainder=False)
dataset_valid.prefetch(2 * batch_size)

<PrefetchDataset shapes: ((?, 18926), (?, 12), (?,), (?, 1)), types: (tf.float32, tf.float32, tf.int32, tf.uint8)>

In [84]:
dataset_train

<BatchDataset shapes: ((?, 18926), (?, 12), (?,), (?, 1)), types: (tf.float32, tf.float32, tf.int32, tf.uint8)>

### tf iterator

In [85]:
## tf.iterator
iterator_train = dataset_train.make_initializable_iterator()
iterator_valid = dataset_valid.make_initializable_iterator()
_ = sess.run(iterator_train.initializer,
                  feed_dict={X_train_PH:X_train,
                             S_train_PH:S_train,
                             I_train_PH:I_train,
                             Y_train_PH:Y_train})
_ = sess.run(iterator_valid.initializer,
                  feed_dict={X_valid_PH:X_valid,
                             S_valid_PH:S_valid,
                             I_valid_PH:I_valid,
                             Y_valid_PH:Y_valid})

In [86]:
## tf.iterator.handle
handle_train = sess.run(iterator_train.string_handle())
handle_valid = sess.run(iterator_valid.string_handle())

### tf graph

In [87]:
## tf.graph
_output_types = dataset_train.output_types
_output_shapes = dataset_train.output_shapes

#create_graph()

### Let's look at what's within the create_graph method of RefDNN!
### Here we can see how the model is built: the layers in it, the loss function, and optimizers, etc.!

In [88]:
#def create_graph():
## tf.Placeholder
handle = tf.placeholder(tf.string, shape=[])
training = tf.placeholder(dtype=bool, name='training_PH')

## tf.Iterator
iterator = tf.data.Iterator.from_string_handle(string_handle=handle,
                                               output_types=_output_types,
                                               output_shapes=_output_shapes)
X_batch, S_batch, I_batch, Y_batch = iterator.get_next()

## Activation function
nonlinear = tf.nn.sigmoid

## Model
dense0    = tf.layers.dense(inputs=X_batch, units=_output_shapes[1][1].value, activation='linear', name='dense0')
activate0 = nonlinear(dense0, name='activation0')

dense1    = tf.layers.dense(inputs=tf.multiply(S_batch, activate0), units=hidden_units, activation='linear', name='dense1')
bn1       = tf.layers.batch_normalization(dense1, training=training, name='bn1')
activate1 = nonlinear(bn1, name='activation1')

dense2    = tf.layers.dense(inputs=activate1, units=hidden_units, activation='linear', name='dense2')
bn2       = tf.layers.batch_normalization(dense2, training=training, name='bn2')
activate2 = nonlinear(bn2, name='activation2')

## Output
O_ELASTICNET  = tf.expand_dims(tf.reduce_sum(tf.multiply(dense0, tf.one_hot(I_batch, depth=_output_shapes[1][1].value)), 1), axis=-1)
O_DNN = tf.layers.dense(inputs=activate2, units=1, activation='linear', name='output')

## LOSS
LOSS_ELASTICNET  = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=O_ELASTICNET, labels=tf.cast(Y_batch, dtype=tf.float32)))
LOSS_DNN = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=O_DNN, labels=tf.cast(Y_batch, dtype=tf.float32)))
LOSS = 0.5 * (LOSS_DNN + LOSS_ELASTICNET)

## OPTIMIZER 1st
OPT_ftrl = tf.train.FtrlOptimizer(learning_rate=learning_rate_ftrl,
                                   l1_regularization_strength=l1_regularization_strength,
                                   l2_regularization_strength=l2_regularization_strength)
var_list_ftrl = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'dense0')

## OPTIMIZER 2nd
OPT_adam = tf.train.AdamOptimizer(learning_rate=learning_rate_adam)
var_list_adam = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'dense1')
var_list_adam += tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'dense2')
var_list_adam += tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'output')
var_list_adam += tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'batchnormalization1')
var_list_adam += tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, 'batchnormalization2')

## MINIMIZATION
update_ops = tf.get_collection(tf.GraphKeys.UPDATE_OPS)
with tf.control_dependencies(update_ops):
    TRAIN_ftrl = OPT_ftrl.minimize(LOSS, var_list=var_list_ftrl)
    TRAIN_adam = OPT_adam.minimize(LOSS, var_list=var_list_adam)

## PREDICTION
P_main = tf.sigmoid(O_DNN)
Y_main = tf.cast(P_main > 0.5, tf.uint8)

## ACCURACY
ACCURACY = tf.metrics.accuracy(labels=Y_batch, predictions=Y_main)
## PRECISION
PRECISION = tf.metrics.precision(labels=Y_batch, predictions=Y_main)
## RECALL
RECALL = tf.metrics.recall(labels=Y_batch, predictions=Y_main)

## SAVE AND RESTORE
saver = tf.train.Saver()

In [89]:
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

In [90]:
## history
history = {
    'train':{
        'loss':[]
    },
    'valid':{
        'loss':[],
        'acc':[],
        'precision':[],
        'recall':[]
    }
}

In [91]:
## initialization for early stopping
earlystop_use=True
if earlystop_use:
    best_loss = np.Inf
    best_step = 0
    cnt_patience = 0

In [92]:
## initialize terminal conditions
termination_convergence = False
termination_earlystop = False

In [93]:
training_steps

1000

There are 1000 training steps here. Let's just try one single trainning step:

In [ ]:
## start fitting
#start_time = time.time()
#for step in range(1, training_steps+1):
#for step in range(1, 1+1):

# just look at one step:

In [94]:
## 1) training 
_, _, loss_train = sess.run([TRAIN_ftrl, TRAIN_adam, LOSS],
                                 feed_dict={handle:handle_train, training:True})

In [95]:
loss_train

2.9284766

We got the loss for this step!  
We will skip the evaluation step here:

In [ ]:
# skip the evaluation step

## 2) evaluation
if step % evaluation_steps == 0:
    ## 2-2) validation loss
    loss_valid = 0.
    acc_valid = 0.
    precision_valid = 0.
    recall_valid = 0.
    n_batch_valid = math.ceil(len(X_valid) // batch_size)
    for _ in range(n_batch_valid):
        metrics_valid = sess.run([LOSS, ACCURACY, PRECISION, RECALL],
                                      feed_dict={handle:handle_valid, training:False})
        loss_valid += metrics_valid[0]
        acc_valid += metrics_valid[1][0]
        precision_valid += metrics_valid[2][0]
        recall_valid += metrics_valid[3][0]
    loss_valid /= n_batch_valid
    acc_valid /= n_batch_valid
    precision_valid /= n_batch_valid
    recall_valid /= n_batch_valid

    ## 2-3) store
    history['train']['loss'].append((step, loss_train))
    history['valid']['loss'].append((step, loss_valid))
    history['valid']['acc'].append((step, acc_valid))
    history['valid']['precision'].append((step, precision_valid))
    history['valid']['recall'].append((step, recall_valid))

    if verbose > 1:
        end_time = time.time()
        log = "[RefDNN] [{:05d}] LOSS_train={:.5f} | LOSS_valid={:.5f}".format(step, loss_train, loss_valid)
        log += " | ACC_valid={:.3f}".format(acc_valid)
        log += " | PRECISION_valid={:.3f}".format(precision_valid)
        log += " | RECALL_valid={:.3f}".format(recall_valid)
        log += " | ({:.3f} sec)".format(end_time-start_time)
        print(log)
        start_time = time.time()

    ## 2-4) early stopping
    if earlystop_use:
        if loss_valid < best_loss:
            best_loss = loss_valid
            best_step = step
            cnt_patience = 0
            saver.save(sess, checkpoint_path)
            if verbose > 1:
                print("[RefDNN] [CHECKPOINT] Model is saved in: {}".format(checkpoint_path))
        elif cnt_patience < patience:
            cnt_patience += 1
        else:
            termination_earlystop = True

    else:
        saver.save(sess, checkpoint_path)
        if verbose > 1:
            print("[RefDNN] [CHECKPOINT] Model is saved in: {}".format(checkpoint_path))

## 3) convergence
if len(history['train']['loss']) > 1:
    termination_convergence = abs(history['train']['loss'][-1][1] - history['train']['loss'][-2][1]) < epsilon

## 4) termination
#if termination_convergence or termination_earlystop:
#    break

### Close the session

In [98]:
## tf.Session
#self._close_session()
sess.close()